# COMP 551 AML Assignment 2
## Ran Cheng, 260768706
This project is for assignment 2 of applied machine learning course.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# read classifier data, mean
ds1_m0_data = pd.read_csv('./Data/DS1_m_0.txt', sep=",", header=None)
ds1_m0_data = ds1_m0_data.dropna(axis=1, how='all')
ds1_m0 = np.squeeze(np.asarray(ds1_m0_data))
ds1_m1_data = pd.read_csv('./Data/DS1_m_1.txt', sep=",", header=None)
ds1_m1_data = ds1_m1_data.dropna(axis=1, how='all')
ds1_m1 = np.squeeze(np.asarray(ds1_m1_data))
print(ds1_m0, ds1_m1)

[1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3 1.3
 1.3 1.3] [2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [3]:
# read covarience matrrix
ds1_cov_data = pd.read_csv('./Data/DS1_Cov.txt', sep=",", header=None)
ds1_cov_data = ds1_cov_data.dropna(axis=1, how='all')
ds1_cov = np.squeeze(np.asarray(ds1_cov_data))
ds1_cov.shape

(20, 20)

In [7]:
# get the data for classifier, 0
out_0 = np.random.multivariate_normal(ds1_m0, ds1_cov, size=2000)
label_0 = np.zeros((2000,1))
labeled_ds1_0 = np.hstack((out_0, label_0))
# get the data for classifier, 1
out_1 = np.random.multivariate_normal(ds1_m1, ds1_cov, size=2000)
label_1 = np.ones((2000,1))
labeled_ds1_1 = np.hstack((out_1, label_1))
print(labeled_ds1_0[0], labeled_ds1_1[1])

[1.46772028 0.80814889 1.78404503 1.2943795  1.34135711 2.81687096
 1.22270178 4.09541582 2.49404259 6.095371   1.48324386 2.84226713
 3.63548691 3.3938471  1.69986612 4.4541586  1.63144761 3.28515225
 1.60744119 3.4854458  0.        ] [1.72869713 2.45825512 2.16915855 2.14944242 2.62383386 2.15532881
 2.36779056 2.34829721 3.27037432 1.71524995 3.09746777 3.27805928
 3.4014519  3.38746775 4.04650658 3.00519198 3.86978212 1.67751204
 4.34960941 3.05262517 1.        ]


(2000, 20)

In [6]:
# concaternate negative and positive data
merged_ds1 = np.vstack((labeled_ds1_0,labeled_ds1_1))
# shuffle the negative and positive data
np.random.shuffle(merged_ds1)
# get the data split into two parts randomly: training and testing data
train, test = train_test_split(merged_ds1, test_size=0.3)
train.shape

(2800, 21)